In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

# ==========================================
# CONFIGURATION
# ==========================================
base_path = "/Users/tilincet/Desktop/INDENG242A/project_Brazil/archive-3"

print("="*70)
print("ENHANCED PIPELINE: MODEL SELECTION -> RECOMMENDATION -> SIMULATION")
print("="*70)

# ==========================================
# 1. LOAD AND PRE-FILTER DATA
# ==========================================
print("\nLOADING DATA...")
files = {
    'customers': 'olist_customers_dataset.csv',
    'orders': 'olist_orders_dataset.csv', 
    'order_items': 'olist_order_items_dataset.csv',
    'order_payments': 'olist_order_payments_dataset.csv',
    'order_reviews': 'olist_order_reviews_dataset.csv',
    'products': 'olist_products_dataset.csv',
    'translation': 'product_category_name_translation.csv'
}

data = {}
for key, file in files.items():
    data[key] = pd.read_csv(f"{base_path}/{file}")
    
df_orders = data['orders'].merge(
    data['customers'][['customer_id', 'customer_unique_id']], on='customer_id', how='inner'
)

cust_counts = df_orders['customer_unique_id'].value_counts()
repeat_customers = cust_counts[cust_counts > 1].index
df_orders = df_orders[df_orders['customer_unique_id'].isin(repeat_customers)]

full_df = df_orders.merge(data['order_reviews'][['order_id', 'review_score']], on='order_id', how='inner')
full_df = full_df.merge(data['order_items'][['order_id', 'product_id', 'price', 'freight_value']], on='order_id', how='inner')
full_df = full_df.merge(data['products'][['product_id', 'product_category_name']], on='product_id', how='left')
full_df = full_df.merge(data['translation'], on='product_category_name', how='left')
full_df['order_purchase_timestamp'] = pd.to_datetime(full_df['order_purchase_timestamp'])
full_df['order_delivered_customer_date'] = pd.to_datetime(full_df['order_delivered_customer_date'], errors='coerce')
full_df['delivery_days'] = (full_df['order_delivered_customer_date'] - full_df['order_purchase_timestamp']).dt.days

# NEW FEATURE 1
category_count = full_df.groupby('order_id')['product_category_name_english'].nunique().reset_index()
category_count.columns = ['order_id', 'category_diversity']
full_df = full_df.merge(category_count, on='order_id', how='left')

# NEW FEATURE 2
full_df['purchase_hour'] = full_df['order_purchase_timestamp'].dt.hour
full_df['is_weekend'] = (full_df['order_purchase_timestamp'].dt.dayofweek >= 5).astype(int)
full_df['purchase_month'] = full_df['order_purchase_timestamp'].dt.month

full_df['high_satisfaction'] = (full_df['review_score'] >= 4).astype(int)

full_df = full_df.dropna(subset=['delivery_days', 'product_category_name_english', 'category_diversity'])

print(f"Final dataset: {len(full_df):,} records")

# ==========================================
# 2. MODEL SELECTION PHASE
# ==========================================
print("\n" + "="*70)
print("PHASE 1: MODEL SELECTION COMPARISON")
print("="*70)

def prepare_model_selection_data(df):
    order_agg = df.groupby('order_id').agg({
        'price': 'sum',
        'freight_value': 'sum', 
        'delivery_days': 'max',
        'category_diversity': 'max',    
        'purchase_hour': 'first',
        'is_weekend': 'first',          
        'purchase_month': 'first',      
        'high_satisfaction': 'max'
    }).reset_index()

    X = order_agg[['price', 'freight_value', 'delivery_days', 
                   'category_diversity', 'purchase_hour', 'is_weekend', 'purchase_month']]
    y = order_agg['high_satisfaction']
    
    return X, y

def compare_classification_models(X, y):

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    models = {
        'LogisticRegression': LogisticRegression(random_state=42, max_iter=1000),
        'DecisionTree': DecisionTreeClassifier(max_depth=10, random_state=42),
        'RandomForest': RandomForestClassifier(n_estimators=100, random_state=42),
        'GradientBoosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
        'AdaBoost': AdaBoostClassifier(n_estimators=100, random_state=42),
        'LinearDiscriminant': LinearDiscriminantAnalysis(),
        'KNN': KNeighborsClassifier(n_neighbors=5)
    }

    results = {}
    print("Model Comparison Results:")
    print("-" * 50)
    
    for name, model in models.items():
        try:
            if name in ['LogisticRegression', 'KNN']:
                model.fit(X_train_scaled, y_train)
                y_pred = model.predict(X_test_scaled)
                y_proba = model.predict_proba(X_test_scaled)[:, 1]
            else:
                model.fit(X_train, y_train)
                y_pred = model.predict(X_test)
                y_proba = model.predict_proba(X_test)[:, 1]

            accuracy = accuracy_score(y_test, y_pred)
            auc = roc_auc_score(y_test, y_proba)
            
            results[name] = {
                'model': model,
                'accuracy': accuracy,
                'auc': auc,
                'scaler': scaler if name in ['LogisticRegression', 'KNN'] else None
            }
            
            print(f"{name:<20}: Accuracy = {accuracy:.4f}, AUC = {auc:.4f}")
            
        except Exception as e:
            print(f"Error with {name}: {e}")
            continue
    
    return results, X_train, X_test, y_train, y_test

X, y = prepare_model_selection_data(full_df)
model_results, X_train, X_test, y_train, y_test = compare_classification_models(X, y)

if model_results:
    best_model_name = max(model_results.items(), key=lambda x: x[1]['accuracy'])[0]
    best_model = model_results[best_model_name]['model']
    best_scaler = model_results[best_model_name]['scaler']
    
    print(f"\n✅ Best Model Selected: {best_model_name}")
    print(f"   Test Accuracy: {model_results[best_model_name]['accuracy']:.4f}")
    print(f"   Test AUC: {model_results[best_model_name]['auc']:.4f}")

    if hasattr(best_model, 'feature_importances_'):
        feature_importance = pd.DataFrame({
            'feature': X.columns,
            'importance': best_model.feature_importances_
        }).sort_values('importance', ascending=False)
        print(f"\n📊 Feature Importance (Top 5):")
        for i, row in feature_importance.head().iterrows():
            print(f"   {row['feature']:<20}: {row['importance']:.4f}")
else:
    print("❌ No models were successfully trained")
    exit()

# ==========================================
# 3. GLOBAL SPLIT (80/20)
# ==========================================
global_train_df, global_test_df = train_test_split(full_df, test_size=0.2, random_state=42)
print(f"\nGLOBAL SPLIT: Train={len(global_train_df):,}, Test={len(global_test_df):,}")

# ==========================================
# 4. STAGE 2: RECOMMENDER WITH BEST MODEL
# ==========================================
print("\n" + "="*70)
print("PHASE 2: RECOMMENDATION WITH BEST MODEL")
print("="*70)

ratings_long = (
    global_train_df[['customer_unique_id', 'product_category_name_english', 'review_score']]
    .dropna()
    .copy()
)

ratings_long.rename(columns={
    'customer_unique_id': 'user',
    'product_category_name_english': 'item', 
    'review_score': 'rating'
}, inplace=True)

users = ratings_long['user'].unique()
items = ratings_long['item'].unique()
user2idx = {u: i for i, u in enumerate(users)}
item2idx = {m: j for j, m in enumerate(items)}
idx2item = {j: m for m, j in item2idx.items()}

n_users, n_items = len(users), len(items)

ratings_long['u_idx'] = ratings_long['user'].map(user2idx)
ratings_long['i_idx'] = ratings_long['item'].map(item2idx)

inner_train, inner_val = train_test_split(ratings_long, test_size=0.2, random_state=42)

user_bias = inner_train.groupby('u_idx')['rating'].mean().reindex(range(n_users)).fillna(0).values
item_bias = inner_train.groupby('i_idx')['rating'].mean().reindex(range(n_items)).fillna(0).values

R_centered = np.zeros((n_users, n_items), dtype=float)
for row in inner_train.itertuples(index=False):
    u, i = int(row.u_idx), int(row.i_idx)
    R_centered[u, i] = row.rating - user_bias[u] - item_bias[i]

def tune_svd(k_list, R_centered, val_df, user_bias, item_bias):
    best_k, best_mae, best_svd, best_Z_hat = None, float('inf'), None, None
    
    for k in k_list:
        svd = TruncatedSVD(n_components=k, random_state=42)
        U = svd.fit_transform(R_centered)
        Z_hat = U @ svd.components_
        
        val_preds, val_actuals = [], []
        for row in val_df.itertuples(index=False):
            u, i = int(row.u_idx), int(row.i_idx)
            if u < Z_hat.shape[0] and i < Z_hat.shape[1]:
                pred = user_bias[u] + item_bias[i] + Z_hat[u, i]
                val_preds.append(pred)
                val_actuals.append(row.rating)
        
        mae = mean_absolute_error(val_actuals, val_preds) if val_preds else float('inf')
        print(f"  k={k}: MAE = {mae:.4f}")
        
        if mae < best_mae:
            best_mae, best_k, best_svd, best_Z_hat = mae, k, svd, Z_hat
    
    return best_k, best_mae, best_svd, best_Z_hat

print("Tuning SVD components...")
k_list = [5, 10, 20, 30]
best_k, best_mae, best_svd, best_Z_hat = tune_svd(k_list, R_centered, inner_val, user_bias, item_bias)
print(f"✓ Best SVD: k={best_k}, MAE={best_mae:.4f}")

# ==========================================
# 5. FINAL SIMULATION WITH BEST MODEL
# ==========================================
print("\n" + "="*70)
print("PHASE 3: SIMULATION WITH OPTIMIZED PIPELINE")
print("="*70)

results = []
test_orders = global_test_df.groupby('order_id')

for i, (order_id, group) in enumerate(test_orders):
    if i >= 1000: 
        break

    price_total = group['price'].sum()
    freight_total = group['freight_value'].sum() 
    days_actual = group['delivery_days'].max()
    customer_unique_id = group['customer_unique_id'].iloc[0]
    actual_satisfaction = group['high_satisfaction'].max()
    
    category_diversity = group['category_diversity'].iloc[0]
    purchase_hour = group['purchase_hour'].iloc[0]
    is_weekend = group['is_weekend'].iloc[0]
    purchase_month = group['purchase_month'].iloc[0]

    features = pd.DataFrame([[price_total, freight_total, days_actual, 
                            category_diversity, purchase_hour, is_weekend, purchase_month]], 
                          columns=['price', 'freight_value', 'delivery_days', 
                                 'category_diversity', 'purchase_hour', 'is_weekend', 'purchase_month'])
    
    if best_scaler is not None:
        features_scaled = best_scaler.transform(features)
        pred_satisfied = best_model.predict(features_scaled)[0]
    else:
        pred_satisfied = best_model.predict(features)[0]
    
    decision, recs = "", []
    
    if pred_satisfied == 1:
        decision = "UPSELL"
        if customer_unique_id in user2idx:
            u_idx = user2idx[customer_unique_id]
            scores = user_bias[u_idx] + item_bias + best_Z_hat[u_idx, :]
            top_indices = np.argsort(scores)[::-1][:3]
            recs = [idx2item[i] for i in top_indices if i in idx2item]
        else:
            decision = "UPSELL (New Profile)"
            recs = ["Trending Items"]
    else:
        decision = "RETENTION" 
        recs = ["Discount Coupon"]
    
    results.append({
        'order_id': order_id,
        'customer_id': customer_unique_id,
        'actual': actual_satisfaction,
        'predicted': pred_satisfied,
        'action': decision,
        'recommendations': recs,
        'model_used': best_model_name,
        'category_diversity': category_diversity,
        'purchase_hour': purchase_hour,
        'is_weekend': is_weekend,
        'purchase_month': purchase_month
    })

# ==========================================
# 6. RESULTS AND EVALUATION
# ==========================================
res_df = pd.DataFrame(results)
accuracy = accuracy_score(res_df['actual'], res_df['predicted'])
print(f"\nFINAL SIMULATION ACCURACY: {accuracy:.4f}")
print(f"Actions Distribution:")
print(res_df['action'].value_counts())

print(f"\n📈 NEW FEATURES ANALYSIS:")
print(f"Average Category Diversity by Action:")
print(res_df.groupby('action')['category_diversity'].mean())
print(f"\nWeekend Purchase Ratio by Action:")
weekend_ratio = res_df.groupby('action')['is_weekend'].mean()
print(weekend_ratio)

res_df.to_csv('enhanced_customer_retention.csv', index=False)
print(f"\n💾 Results saved to 'enhanced_customer_retention.csv'")

ENHANCED PIPELINE: MODEL SELECTION -> RECOMMENDATION -> SIMULATION

LOADING DATA...
Final dataset: 7,795 records

PHASE 1: MODEL SELECTION COMPARISON
Model Comparison Results:
--------------------------------------------------
LogisticRegression  : Accuracy = 0.8185, AUC = 0.6282
DecisionTree        : Accuracy = 0.7940, AUC = 0.5473
RandomForest        : Accuracy = 0.8230, AUC = 0.6512
GradientBoosting    : Accuracy = 0.8247, AUC = 0.6238
AdaBoost            : Accuracy = 0.8169, AUC = 0.5946
LinearDiscriminant  : Accuracy = 0.8169, AUC = 0.6274
KNN                 : Accuracy = 0.8029, AUC = 0.6292

✅ Best Model Selected: GradientBoosting
   Test Accuracy: 0.8247
   Test AUC: 0.6238

📊 Feature Importance (Top 5):
   delivery_days       : 0.5141
   freight_value       : 0.2119
   price               : 0.1316
   purchase_month      : 0.0565
   purchase_hour       : 0.0503

GLOBAL SPLIT: Train=6,236, Test=1,559

PHASE 2: RECOMMENDATION WITH BEST MODEL
Tuning SVD components...
  k=5: MAE = 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import accuracy_score, mean_absolute_error

# ==========================================
# CONFIGURATION
# ==========================================
base_path = "."   # Update to your data path

print("="*70)
print("PIPELINE: REPEAT CUSTOMERS ONLY -> TRAIN -> SIMULATE")
print("="*70)

# ==========================================
# 1. LOAD AND PRE-FILTER DATA
# ==========================================
print("\nLOADING DATA...")
files = {
    'customers': 'olist_customers_dataset.csv',
    'orders': 'olist_orders_dataset.csv',
    'order_items': 'olist_order_items_dataset.csv',
    'order_reviews': 'olist_order_reviews_dataset.csv',
    'products': 'olist_products_dataset.csv',
    'translation': 'product_category_name_translation.csv'
}

data = {}
for key, file in files.items():
    data[key] = pd.read_csv(f"{base_path}/{file}")

# --- CRITICAL STEP: MAP TO UNIQUE CUSTOMER ID ---
# In Olist, 'customer_id' changes every order. 'customer_unique_id' is the real user.
df_orders = data['orders'].merge(
    data['customers'][['customer_id', 'customer_unique_id']], 
    on='customer_id', 
    how='inner'
)

# --- FILTER: KEEP ONLY REPEAT CUSTOMERS ---
print("\nFILTERING FOR REPEAT BUYERS...")
cust_counts = df_orders['customer_unique_id'].value_counts()
repeat_customers = cust_counts[cust_counts > 1].index

initial_count = len(df_orders)
df_orders = df_orders[df_orders['customer_unique_id'].isin(repeat_customers)]
filtered_count = len(df_orders)

print(f"  Original Orders: {initial_count:,}")
print(f"  Filtered Orders: {filtered_count:,} (Belonging to users with 2+ orders)")
print(f"  Unique Repeat Users: {df_orders['customer_unique_id'].nunique():,}")

# Merge remaining datasets (Items, Reviews, Products)
full_df = df_orders.merge(
    data['order_reviews'][['order_id', 'review_score']], 
    on='order_id', how='inner'
)
full_df = full_df.merge(
    data['order_items'][['order_id', 'product_id', 'price', 'freight_value']], 
    on='order_id', how='inner'
)
full_df = full_df.merge(
    data['products'][['product_id', 'product_category_name']], 
    on='product_id', how='left'
)
full_df = full_df.merge(
    data['translation'], 
    on='product_category_name', how='left'
)

# Feature Engineering 【revision】
full_df['order_purchase_timestamp'] = pd.to_datetime(full_df['order_purchase_timestamp'])
full_df['order_delivered_customer_date'] = pd.to_datetime(
    full_df['order_delivered_customer_date'], errors='coerce'
)
full_df['delivery_days'] = (
    full_df['order_delivered_customer_date'] - full_df['order_purchase_timestamp']
).dt.days

# Cleanup
full_df = full_df.dropna(subset=['delivery_days', 'product_category_name_english'])
full_df['high_satisfaction'] = (full_df['review_score'] >= 4).astype(int)

print(f"Total records after merge & cleanup: {len(full_df):,}")

# ==========================================
# 2. GLOBAL SPLIT (80/20)
# ==========================================
# We split by ORDER, but since everyone has >1 order, 
# it is highly likely a user appears in both sets.
global_train_df, global_test_df = train_test_split(full_df, test_size=0.2, random_state=42)

print(f"\nGLOBAL SPLIT:")
print(f"✓ Training Set: {len(global_train_df):,}")
print(f"✓ Test Set:     {len(global_test_df):,}")

# ==========================================
# 3. STAGE 1: SATISFACTION MODEL
# ==========================================
print("\n[Stage 1] Training Satisfaction Model...")

# Aggregate by order for classification features
train_agg = global_train_df.groupby('order_id').agg({
    'price': 'sum',
    'freight_value': 'sum',
    'delivery_days': 'max',
    'high_satisfaction': 'max'
}).reset_index()

X_train_sat = train_agg[['price', 'freight_value', 'delivery_days']]
y_train_sat = train_agg['high_satisfaction']

clf = RandomForestClassifier(n_estimators=50, max_depth=10, random_state=42)
clf.fit(X_train_sat, y_train_sat)
print("✓ Classifier Trained")

# ==========================================
# 4. STAGE 2: RECOMMENDER (TUNING K)
# ==========================================
print("\n[Stage 2] Training SVD Recommender on REPEAT USERS...")

# Use 'customer_unique_id' as 'user' now!
ratings_long = (
    global_train_df[['customer_unique_id', 'product_category_name_english', 'review_score']]
    .dropna()
    .copy()
)

ratings_long.rename(columns={
    'customer_unique_id': 'user',  # CHANGED FROM customer_id
    'product_category_name_english': 'item',
    'review_score': 'rating'
}, inplace=True)

# Map Indices
users = ratings_long['user'].unique()
items = ratings_long['item'].unique()

user2idx = {u: i for i, u in enumerate(users)}
item2idx = {m: j for j, m in enumerate(items)}
idx2item = {j: m for m, j in item2idx.items()}

n_users = len(users)
n_items = len(items)

ratings_long['u_idx'] = ratings_long['user'].map(user2idx)
ratings_long['i_idx'] = ratings_long['item'].map(item2idx)

# Inner Split for Tuning
inner_train, inner_val = train_test_split(ratings_long, test_size=0.2, random_state=42)

# Calculate Biases
user_bias_s = inner_train.groupby('u_idx')['rating'].mean()
item_bias_s = inner_train.groupby('i_idx')['rating'].mean()
user_bias = user_bias_s.reindex(range(n_users)).fillna(0).values 
item_bias = item_bias_s.reindex(range(n_items)).fillna(0).values 

# Center Matrix
print("Constructing Centered Matrix...")
R_centered = np.zeros((n_users, n_items), dtype=float)
for row in inner_train.itertuples(index=False):
    u, i = int(row.u_idx), int(row.i_idx)
    R_centered[u, i] = row.rating - user_bias[u] - item_bias[i]

# Tuning Function
def train_svd_and_eval(k, R_centered, val_df, user_bias, item_bias):
    svd = TruncatedSVD(n_components=k, random_state=42)
    U = svd.fit_transform(R_centered)
    V = svd.components_
    Z_hat = U @ V 

    val_preds = []
    val_actuals = []
    for row in val_df.itertuples(index=False):
        u, i = int(row.u_idx), int(row.i_idx)
        # Handle cases where user/item might be missing from inner split
        if u < Z_hat.shape[0] and i < Z_hat.shape[1]:
            pred = user_bias[u] + item_bias[i] + Z_hat[u, i]
            val_preds.append(pred)
            val_actuals.append(row.rating)
            
    if not val_preds: return svd, Z_hat, float('inf')
    return svd, Z_hat, mean_absolute_error(val_actuals, val_preds)

# Run Tuning
k_list = [5, 10, 20, 30] 
best_k = None
best_mae = float('inf')
best_Z_hat = None

print(f"Tuning k over: {k_list}")
for k in k_list:
    svd, Z_hat, mae_val = train_svd_and_eval(k, R_centered, inner_val, user_bias, item_bias)
    print(f"  k={k}: MAE = {mae_val:.4f}")
    if mae_val < best_mae:
        best_mae = mae_val
        best_k = k
        best_Z_hat = Z_hat

print(f"✓ Best model: k={best_k}")

# ==========================================
# 5. STAGE 3: SIMULATION (ON TEST SET)
# ==========================================
print("\n" + "="*70)
print("RUNNING SIMULATION ON REPEAT CUSTOMERS TEST SET")
print("="*70)

results = []
test_orders = global_test_df.groupby('order_id')
counter = 0

for order_id, group in test_orders:
    counter += 1
    
    # Extract Context
    price_total = group['price'].sum()
    freight_total = group['freight_value'].sum()
    days_actual = group['delivery_days'].max()
    # IMPORTANT: Use unique ID here too
    customer_unique_id = group['customer_unique_id'].iloc[0] 
    actual_satisfaction = group['high_satisfaction'].max()

    # Predict Satisfaction
    features = pd.DataFrame([[price_total, freight_total, days_actual]], 
                            columns=['price', 'freight_value', 'delivery_days'])
    pred_satisfied = clf.predict(features)[0]

    decision = ""
    recs = []
    
    if pred_satisfied == 1:
        decision = "UPSELL"
        
        # Check if customer_unique_id exists in our training user map
        if customer_unique_id in user2idx:
            u_idx = user2idx[customer_unique_id]
            scores = user_bias[u_idx] + item_bias + best_Z_hat[u_idx, :]
            top_indices = np.argsort(scores)[::-1][:3]
            recs = [idx2item[i] for i in top_indices]
        else:
            # Only happens if ALL of this user's orders ended up in Test (unlikely for heavy users)
            decision = "UPSELL (New Profile)" 
            recs = ["Trending Items"]
    else:
        decision = "RETENTION"
        recs = ["Discount Coupon"]

    results.append({
        'order_id': order_id,
        'customer_id': customer_unique_id, # Log the unique ID
        'actual': actual_satisfaction,
        'predicted': pred_satisfied,
        'action': decision,
        'recommendations': recs
    })
    
    if counter >= 1000: break

# ==========================================
# 6. RESULTS
# ==========================================
res_df = pd.DataFrame(results)

res_df.to_csv('customer_retent.csv', index=False)

PIPELINE: REPEAT CUSTOMERS ONLY -> TRAIN -> SIMULATE

LOADING DATA...

FILTERING FOR REPEAT BUYERS...
  Original Orders: 99,441
  Filtered Orders: 6,342 (Belonging to users with 2+ orders)
  Unique Repeat Users: 2,997
Total records after merge & cleanup: 7,795

GLOBAL SPLIT:
✓ Training Set: 6,236
✓ Test Set:     1,559

[Stage 1] Training Satisfaction Model...
✓ Classifier Trained

[Stage 2] Training SVD Recommender on REPEAT USERS...
Constructing Centered Matrix...
Tuning k over: [5, 10, 20, 30]
  k=5: MAE = 2.3473
  k=10: MAE = 1.9654
  k=20: MAE = 1.7091
  k=30: MAE = 1.6140
✓ Best model: k=30

RUNNING SIMULATION ON REPEAT CUSTOMERS TEST SET
